<a href="https://colab.research.google.com/github/AlexxxSem111/quest/blob/main/quest_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
device = 'cuda'
step_count = 0
user_count = 0
binary_steps = [0, 1, 3]

# Данные для логики квеста
Их немного у меня поэтому оставил так.
Здесь нет суммаризации прошлых ответов но их не трудно добавить, просто думаю это уже у вас реализовано как то по другому.

Они содержат номер Шага, варианты запроса для Шага, варианты следующего Шага и возможна ли пользовательская Игра на Шаге.
Колонка 'Action' содержит дополнительные инструкции к промпту.

Ход этого квеста:
- Шаг 0 - должно быть(в деплое) приветствие с вопросом "хотите пройти квест?" и первый запрос это ответ на этот вопрос.
- Шаг 1 - короткая история с вопросом "хотите продолжить?"
- Шаг 2 - история в которой есть развилка и нужно выбрать куда пойти. (кластеризацию запроса для этого шага я пока не делал)
    чтобы протестировать нужно, чтобы запрос обязательно содержал одно из этих 3 слов:
    - 'right' - 1 вариант истории.
    - 'left' - 2 вариант истории.
    - 'any' - свой вариант, история меняется Шаг остается 2, после нескольких своих вариантов Шаг автоматически увеличится.
- Шаг 3 - история с вопросом "что дальше?" в зависимости от ответа вариант истории и сброс на 0.

In [ ]:
import pandas as pd

data = [{'Step': 0,
  'Variant': 'POS',
  'Action': 'Tell a short 4-5 sentence fairy tale about a bear. Finish story with variant of question "Do you want to continue?".',
  'ToStep': 1,
  'UserGame': False},
 {'Step': 0,
  'Variant': 'NEG',
  'Action': 'Say something with variant of "Thank you, goodbye".',
  'ToStep': 0,
  'UserGame': False},
 {'Step': 1,
  'Variant': 'POS',
  'Action': 'Tell a short 4-5 sentence fairy tale about a bear at the fork of road. Finish story with variant of question "Which way?".',
  'ToStep': 2,
  'UserGame': False},
 {'Step': 1,
  'Variant': 'NEG',
  'Action': 'Tell a short 1-2 sentence fairy tale about a bear dying. Finish story with variant of question "Do you want to try again?".',
  'ToStep': 0,
  'UserGame': False},
 {'Step': 2,
  'Variant': 'Right',
  'Action': 'Tell a short 4-5 sentence fairy tale about a bear visit a rabbit. Finish story with variant of question "What s next?"',
  'ToStep': 3,
  'UserGame': False},
 {'Step': 2,
  'Variant': 'Left',
  'Action': 'Tell a short 2-3 sentence fairy tale about a bear dying. Finish story with variant of question "Do you want to try again?".',
  'ToStep': 0,
  'UserGame': False},
 {'Step': 2,
  'Variant': 'Any',
  'Action': 'Tell a short 7-8 sentence fairy tale about a bear at fork of road. Finish story with variant of question "Which way?".',
  'ToStep': 2,
  'UserGame': True},
 {'Step': 3,
  'Variant': 'POS',
  'Action': 'Tell a short 4-5 sentence fairy tale about a bear with happy end. Finish story with congratulations.',
  'ToStep': 0,
  'UserGame': False},
 {'Step': 3,
  'Variant': 'NEG',
  'Action': 'Tell a short 2-3 sentence fairy tale about a bear with bad end. Finish story with congratulations.',
  'ToStep': 0,
  'UserGame': False}]

df = pd.DataFrame(data)
df

,Step,Variant,Action,ToStep,UserGame
0,0,POS,Tell a short 4-5 sentence fairy tale about a b...,1,False
1,0,NEG,"Say something with variant of ""Thank you, good...",0,False
2,1,POS,Tell a short 4-5 sentence fairy tale about a b...,2,False
3,1,NEG,Tell a short 1-2 sentence fairy tale about a b...,0,False
4,2,Right,Tell a short 4-5 sentence fairy tale about a b...,3,False
5,2,Left,Tell a short 2-3 sentence fairy tale about a b...,0,False
6,2,Any,Tell a short 7-8 sentence fairy tale about a b...,2,True
7,3,POS,Tell a short 4-5 sentence fairy tale about a b...,0,False
8,3,NEG,Tell a short 2-3 sentence fairy tale about a b...,0,False


# Модель для sentiment-analysis
Натренирована мной на 4000 комментариях из отзывов IMBD, выложена на HuggingFace.

В binary_steps шагах определяет: положительный или отрицательный запрос.

In [ ]:
from transformers import (
    pipeline,
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast
)

sentiment_model_path = 'AlexxxSem/distilbert-yes-no'

sentiment_model = DistilBertForSequenceClassification.from_pretrained(sentiment_model_path)
sentiment_tokenizer = DistilBertTokenizerFast.from_pretrained(sentiment_model_path)

sentiment = pipeline("sentiment-analysis",
                     model=sentiment_model,
                     tokenizer=sentiment_tokenizer)

# Функция обработки квеста и модификации запроса

In [ ]:
def modify_query(query, df):
    global step_count
    global user_count

    # Фильтруем DataFrame по текущему step_count
    step_df = df[df['Step'] == step_count]

    if step_count in binary_steps:
        # Определяем 'sentiment' запроса
        desired_value = sentiment(query)[0]['label']

        # Выбираем строку, где 'Variant' равен 'sentiment'
        selected_row = step_df[step_df['Variant'] == desired_value]

        # Получаем значение (Action, ToStep, UserGame)
        action = selected_row['Action'].values[0]
        to_step = selected_row['ToStep'].values[0]
        user_game = selected_row['UserGame'].values[0]

    else:
        # Получаем уникальные варианты для данного шага
        variants = step_df['Variant'].unique()

        for variant in variants:
            if variant.lower() in query.lower():
                # Получаем значение (Action, ToStep, UserGame) для варианта
                action = step_df[step_df['Variant'] == variant]['Action'].values[0]
                to_step = step_df[step_df['Variant'] == variant]['ToStep'].values[0]
                user_game = step_df[step_df['Variant'] == variant]['UserGame'].values[0]

    # Изменяем запрос
    instruction = ' Use following instruction: '

    query = action + instruction + query

    # Меняем шаг
    step_count = to_step

    # Обработка UserGame
    if user_count == 2 and to_step == step_count:
        user_count = 0
        step_count += 1
    elif user_count != 0 and to_step != step_count:
        user_count = 0

    if user_game:
        user_count +=1

    return query

# Основная модель

In [ ]:
!pip install -q -U bitsandbytes accelerate

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## Настройки для работы в Colab

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    device_map="auto",
)

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Декоратор

In [ ]:
def modify_query_wrapper(func):
    def wrapper(query):
        modified_query = modify_query(query, df)
        return func(modified_query)
    return wrapper

# Функция запроса

In [ ]:
import re

@modify_query_wrapper
def generate_response(query):
    messages = [
        {"role": "user", "content": query}
    ]

    input_encoding = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

    generated_ids = model.generate(input_encoding,
                                   pad_token_id=tokenizer.eos_token_id,
                                   max_new_tokens=1000,
                                   do_sample=True)

    decoded_response = tokenizer.batch_decode(generated_ids,
                                              skip_special_tokens=True)

    #decoded_result = re.sub(r'\[INST\].*?\[/INST\] ', '', decoded_response[0])
    decoded_result = decoded_response[0].replace("[INST]", "Query:\n\n").replace("[/INST]", "\n\nStory:\n\n")

    return decoded_result

# Тестирование

In [ ]:
# Для быстрого сброса)
step_count = 0
user_count = 0

In [ ]:
print(step_count)
print(user_count)

0
1


In [ ]:
message = "go"

print(generate_response(message))

Query:

 Tell a short 2-3 sentence fairy tale about a bear with bad end. Finish story with congratulations. Use following instruction: any 

Story:

 Once upon a time, there was a bear who lived happily in the forest, but as time went by, he grew tired of his simple life and yearned for more. He made his way to a nearby town, where he was met with cheers and applause, but eventually, the bear's greed and power-hunger consumed him, and he turned into a cruel and ruthless leader. Congratulations on finishing the fairy tale!


# Красивое тестирование
У меня почему то сразу уменьшается время бесплатного GPU в Colab

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

def response(message, history):
    return generate_response(message)

gr.ChatInterface(response, title='Do you want play Quest?').launch()